[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/raghumriyer/working/blob/master/Assignment_4_DNST_CIFAR10_v11.ipynb)

In [0]:
# https://keras.io/
!pip install -q keras
import keras

In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator


In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 40
num_filter = 12
compression = 0.5
dropout_rate = 0.2

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)



In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 12
dropout_rate = 0.2
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [35]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_105 (Conv2D)             (None, 32, 32, 12)   324         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_105 (BatchN (None, 32, 32, 12)   48          conv2d_105[0][0]                 
__________________________________________________________________________________________________
activation_105 (Activation)     (None, 32, 32, 12)   0           batch_normalization_105[0][0]    
__________________________________________________________________________________________________
conv2d_106

In [0]:
#data augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(x_train)

  
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              #optimizer=Adam(),
              optimizer=SGD(lr=0.01, momentum=0.9, decay=0.0001, nesterov=False),
              metrics=['accuracy'])

In [44]:
#model.fit(x_train, y_train,
                    #batch_size=batch_size,
                    #epochs=epochs,
                    #verbose=1,
                    #validation_data=(x_test, y_test))

 # Fit the model on the batches generated by datagen.flow().
 #   model.fit_generator(datagen.flow(x_train, y_train,
 #                                    batch_size=batch_size),
 #                       epochs=epochs,
 #                       verbose=1,
 #                       validation_data=(x_test, y_test),
 #                       workers=4)
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),\
                    steps_per_epoch=x_train.shape[0] // batch_size,epochs=100,\
                    verbose=1,validation_data=(x_test,y_test))
                    #,callbacks=[LearningRateScheduler(lr_schedule)])



Epoch 1/100
390/390 [==============================] - 118s 303ms/step - loss: 0.8702 - acc: 0.6889 - val_loss: 1.3411 - val_acc: 0.6220
Epoch 2/100
390/390 [==============================] - 119s 304ms/step - loss: 0.8554 - acc: 0.6943 - val_loss: 1.2854 - val_acc: 0.6400
Epoch 3/100
390/390 [==============================] - 118s 301ms/step - loss: 0.8406 - acc: 0.6979 - val_loss: 1.4431 - val_acc: 0.6272
Epoch 4/100
390/390 [==============================] - 120s 308ms/step - loss: 0.8308 - acc: 0.7034 - val_loss: 1.5340 - val_acc: 0.6185
Epoch 5/100
390/390 [==============================] - 121s 310ms/step - loss: 0.8209 - acc: 0.7059 - val_loss: 1.5144 - val_acc: 0.6255
Epoch 6/100
390/390 [==============================] - 118s 302ms/step - loss: 0.8158 - acc: 0.7082 - val_loss: 1.2228 - val_acc: 0.6573
Epoch 7/100
390/390 [==============================] - 118s 303ms/step - loss: 0.8078 - acc: 0.7132 - val_loss: 1.1995 - val_acc: 0.6600
Epoch 8/100
390/390 [====================

In [45]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 12s 1ms/step
Test loss: 1.013374768257141
Test accuracy: 0.7321


In [46]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model.h5')

MessageError: ignored